# Mini-projeto 06 — PyArrow & Parquet

Persistência eficiente: parquet e particionamento.

> Entregável de portfólio — gerado em 2026-01-30.


## Objetivo

        Escrever e ler Parquet com PyArrow e criar um dataset particionado por `region`.

        ## Entregáveis (para portfólio)

        - [ ] `outputs/sales.parquet`
- [ ] `outputs/sales_ds/` (particionado)
- [ ] `reports/parquet_notes.md`

        ## Como usar

        1- Rode este notebook (kernel com o `.venv` do repo)  
        2- Gere **assets** (imagens/HTML) e **reports** (markdown/json) dentro desta pasta  
        3- Faça commit dos arquivos gerados para evidenciar o resultado no GitHub

In [ ]:
# Setup: detectar raiz do repositório + paths padrão
from pathlib import Path
import pandas as pd
import numpy as np

def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for _ in range(12):
        if (p / "requirements.txt").exists() and (p / "README.md").exists():
            return p
        p = p.parent
    return (start or Path.cwd()).resolve()

ROOT = find_repo_root()
DATA_SAMPLE = ROOT / "data" / "sample"
DATA_SOURCE = ROOT / "data" / "source" / "bases-dados-analytics-powerbi-ml"

# Pasta do projeto (onde salvar assets/outputs/reports)
PROJ = ROOT / "projects" / "06_pyarrow_parquet"
ASSETS = PROJ / "assets"
OUTPUTS = PROJ / "outputs"
REPORTS = PROJ / "reports"
for d in (ASSETS, OUTPUTS, REPORTS):
    d.mkdir(parents=True, exist_ok=True)

# Dataset padrão (sempre disponível)
sales_path = DATA_SAMPLE / "sales.csv"
customers_path = DATA_SAMPLE / "customers.csv"
sales = pd.read_csv(sales_path, parse_dates=["date"])
customers = pd.read_csv(customers_path, parse_dates=["signup_date"])

# Dataset real (opcional): se você adicionou o submodule/clone em dados/source/
has_real = DATA_SOURCE.exists()

print("ROOT:", ROOT)
print("Dataset sample:", sales.shape, customers.shape)
print("Dataset real disponível?", has_real)

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# DataFrame -> Arrow Table
table = pa.Table.from_pandas(sales)

out_file = OUTPUTS / "sales.parquet"
pq.write_table(table, out_file)
print("Salvo:", out_file)

# Dataset particionado
out_ds = OUTPUTS / "sales_ds"
pq.write_to_dataset(table, root_path=str(out_ds), partition_cols=["region"])
print("Dataset particionado em:", out_ds)

# Ler de volta via dataset (scan)
dataset = ds.dataset(str(out_ds), format="parquet")
# Filtrar uma região
sudeste = dataset.to_table(filter=ds.field("region") == "Sudeste").to_pandas()
display(sudeste.head())

md = []
md.append("# Notas Parquet — 06_pyarrow_parquet\n")
md.append("- Parquet melhora leitura/armazenamento em datasets tabulares\n")
md.append("- Particionar ajuda em filtros (pruning)\n")
md.append(f"- Exemplo: Sudeste linhas = {len(sudeste)}\n")
out = REPORTS / "parquet_notes.md"
out.write_text("\n".join(md), encoding="utf-8")
print("Salvo:", out)